# AI Trading Assistant - Strategy Backtest Analysis

**Date:** 2025-11-28 (Created) | To be executed: 2025-11-29

**Objective:** Evaluate VCP and Pivot strategies with focus on:
1. Higher trade frequency with R:R ≥ 3.0
2. VCP contraction proximity effect analysis
3. Fixed target vs trailing stop comparison

---

## 1. Setup and Imports

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add backtest folder to path
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print(f"Backtest started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Import backtest framework
from backtest_framework_v2 import DataCache, Backtester, StrategyScanner, VCPAnalyzer, print_results
from strategy_params import VCP_PARAMS, PIVOT_PARAMS, RISK_PARAMS

print("Framework loaded successfully")

## 2. Test Universe Definition

In [ ]:
# 60 US Stocks across sectors
TEST_SYMBOLS = [
    # Tech Leaders (20)
    'AAPL', 'MSFT', 'NVDA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'AMD', 'AVGO', 'CRM',
    'NFLX', 'ADBE', 'NOW', 'PANW', 'SNOW', 'DDOG', 'NET', 'CRWD', 'ZS', 'MDB',
    # Healthcare (10)
    'LLY', 'UNH', 'JNJ', 'ABBV', 'MRK', 'PFE', 'ISRG', 'VRTX', 'REGN', 'GILD',
    # Industrials (10)
    'GE', 'CAT', 'BA', 'UNP', 'HON', 'RTX', 'DE', 'LMT', 'MMM', 'FDX',
    # Financials (10)
    'JPM', 'V', 'MA', 'BAC', 'GS', 'MS', 'BLK', 'AXP', 'C', 'WFC',
    # Consumer (10)
    'HD', 'LOW', 'NKE', 'SBUX', 'MCD', 'DIS', 'COST', 'WMT', 'TGT', 'LULU',
]

# Display as DataFrame
symbols_df = pd.DataFrame({
    'Sector': ['Tech']*20 + ['Healthcare']*10 + ['Industrials']*10 + ['Financials']*10 + ['Consumer']*10,
    'Symbol': TEST_SYMBOLS
})

print(f"Test Universe: {len(TEST_SYMBOLS)} symbols")
print(f"\nSector Distribution:")
display(symbols_df.groupby('Sector').count().rename(columns={'Symbol': 'Count'}))

## 3. Initialize Data Cache and Load Historical Data

In [ ]:
# Initialize cache
cache_dir = '../cache'
data_cache = DataCache(cache_dir)

# Load data for all symbols (this may take a few minutes on first run)
print("Loading historical data...")
data_status = []

for symbol in TEST_SYMBOLS:
    df = data_cache.get_historical_data(symbol, '2y')
    if df is not None:
        data_status.append({
            'Symbol': symbol,
            'Records': len(df),
            'Start': df.index[0].strftime('%Y-%m-%d'),
            'End': df.index[-1].strftime('%Y-%m-%d'),
            'Status': 'OK'
        })
    else:
        data_status.append({
            'Symbol': symbol,
            'Records': 0,
            'Start': '-',
            'End': '-',
            'Status': 'FAILED'
        })

data_status_df = pd.DataFrame(data_status)
print(f"\nData loaded: {len(data_status_df[data_status_df['Status']=='OK'])}/{len(TEST_SYMBOLS)} symbols")
display(data_status_df.head(10))

## 4. Current Strategy Parameters

In [ ]:
# Display current VCP parameters
print("=" * 60)
print("VCP Strategy Parameters")
print("=" * 60)
vcp_params_df = pd.DataFrame([
    {'Parameter': k, 'Value': v, 'Description': ''} 
    for k, v in VCP_PARAMS['current'].items()
])
display(vcp_params_df)

print("\n" + "=" * 60)
print("Pivot Strategy Parameters")
print("=" * 60)
pivot_params_df = pd.DataFrame([
    {'Parameter': k, 'Value': v} 
    for k, v in PIVOT_PARAMS['current'].items()
])
display(pivot_params_df)

print("\n" + "=" * 60)
print("Risk Management Parameters")
print("=" * 60)
risk_params_df = pd.DataFrame([
    {'Parameter': k, 'Value': v} 
    for k, v in RISK_PARAMS['current'].items()
])
display(risk_params_df)

---
# PHASE 1: Baseline Tests
---

## 5. VCP Baseline Backtest

In [ ]:
# Initialize backtester
backtester = Backtester(data_cache)

# Run VCP baseline
print("Running VCP Baseline Backtest...")
vcp_baseline = backtester.run_backtest(
    symbols=TEST_SYMBOLS,
    strategy_name="VCP_Baseline",
    params=VCP_PARAMS['current'],
    risk_params=RISK_PARAMS['current'],
    exit_method='fixed_target',
    start_date='2023-01-01'
)

print_results(vcp_baseline)

In [ ]:
# VCP Baseline Trades Table
if vcp_baseline.trades:
    trades_df = pd.DataFrame([{
        'Symbol': t.symbol,
        'Entry Date': t.entry_date,
        'Entry Price': t.entry_price,
        'Exit Date': t.exit_date,
        'Exit Price': t.exit_price,
        'Exit Reason': t.exit_reason,
        'P&L %': t.pnl_pct,
        'Days Held': t.days_held,
        'RS Rating': t.rs_rating,
        'Proximity': t.proximity_score,
    } for t in vcp_baseline.trades])
    
    print(f"VCP Baseline: {len(trades_df)} trades")
    display(trades_df.head(20))
else:
    print("No trades generated")

In [ ]:
# VCP Baseline P&L Distribution
if vcp_baseline.trades:
    pnl_values = [t.pnl_pct for t in vcp_baseline.trades if t.pnl_pct is not None]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(pnl_values, bins=20, edgecolor='black', alpha=0.7)
    axes[0].axvline(x=0, color='red', linestyle='--', label='Breakeven')
    axes[0].axvline(x=np.mean(pnl_values), color='green', linestyle='--', label=f'Mean: {np.mean(pnl_values):.1f}%')
    axes[0].set_xlabel('P&L %')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('VCP Baseline - P&L Distribution')
    axes[0].legend()
    
    # Exit Reason Pie Chart
    exit_reasons = pd.Series([t.exit_reason for t in vcp_baseline.trades]).value_counts()
    axes[1].pie(exit_reasons.values, labels=exit_reasons.index, autopct='%1.1f%%')
    axes[1].set_title('Exit Reasons')
    
    plt.tight_layout()
    plt.show()

## 6. Pivot Baseline Backtest

In [ ]:
# Run Pivot baseline
print("Running Pivot Baseline Backtest...")
pivot_baseline = backtester.run_backtest(
    symbols=TEST_SYMBOLS,
    strategy_name="Pivot_Baseline",
    params=PIVOT_PARAMS['current'],
    risk_params=RISK_PARAMS['current'],
    exit_method='fixed_target',
    start_date='2023-01-01'
)

print_results(pivot_baseline)

In [ ]:
# Baseline Comparison Table
comparison_df = pd.DataFrame([
    {
        'Strategy': 'VCP Baseline',
        'Total Trades': vcp_baseline.total_trades,
        'Win Rate %': vcp_baseline.win_rate,
        'Avg Win %': vcp_baseline.avg_win_pct,
        'Avg Loss %': vcp_baseline.avg_loss_pct,
        'Profit Factor': vcp_baseline.profit_factor,
        'Avg R:R': vcp_baseline.avg_rr_realized,
        'Total Return %': vcp_baseline.total_return_pct,
        'Avg Days': vcp_baseline.avg_days_held,
    },
    {
        'Strategy': 'Pivot Baseline',
        'Total Trades': pivot_baseline.total_trades,
        'Win Rate %': pivot_baseline.win_rate,
        'Avg Win %': pivot_baseline.avg_win_pct,
        'Avg Loss %': pivot_baseline.avg_loss_pct,
        'Profit Factor': pivot_baseline.profit_factor,
        'Avg R:R': pivot_baseline.avg_rr_realized,
        'Total Return %': pivot_baseline.total_return_pct,
        'Avg Days': pivot_baseline.avg_days_held,
    }
])

print("=" * 80)
print("BASELINE COMPARISON")
print("=" * 80)
display(comparison_df)

---
# PHASE 2: VCP Contraction Proximity Analysis
---

## 7. Proximity Score Distribution

In [ ]:
# Analyze proximity scores from VCP baseline
if vcp_baseline.trades:
    prox_data = [(t.proximity_score, t.pnl_pct, t.exit_reason) 
                 for t in vcp_baseline.trades 
                 if t.proximity_score is not None and t.pnl_pct is not None]
    
    if prox_data:
        prox_df = pd.DataFrame(prox_data, columns=['Proximity', 'PnL', 'Exit'])
        prox_df['Win'] = prox_df['PnL'] > 0
        
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
        
        # Proximity distribution
        axes[0].hist(prox_df['Proximity'], bins=20, edgecolor='black', alpha=0.7)
        axes[0].set_xlabel('Proximity Score')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title('Proximity Score Distribution')
        
        # Proximity vs P&L scatter
        colors = ['green' if w else 'red' for w in prox_df['Win']]
        axes[1].scatter(prox_df['Proximity'], prox_df['PnL'], c=colors, alpha=0.6)
        axes[1].axhline(y=0, color='black', linestyle='--')
        axes[1].set_xlabel('Proximity Score')
        axes[1].set_ylabel('P&L %')
        axes[1].set_title('Proximity vs P&L')
        
        # Win rate by proximity bucket
        prox_df['Bucket'] = pd.cut(prox_df['Proximity'], bins=[0, 30, 50, 70, 100], labels=['0-30', '30-50', '50-70', '70-100'])
        win_by_bucket = prox_df.groupby('Bucket')['Win'].mean() * 100
        win_by_bucket.plot(kind='bar', ax=axes[2], color='steelblue', edgecolor='black')
        axes[2].set_xlabel('Proximity Bucket')
        axes[2].set_ylabel('Win Rate %')
        axes[2].set_title('Win Rate by Proximity Bucket')
        axes[2].tick_params(axis='x', rotation=0)
        
        plt.tight_layout()
        plt.show()
        
        # Stats table
        print("\nProximity Analysis:")
        print(f"  Correlation (Proximity, Win): {prox_df['Proximity'].corr(prox_df['Win'].astype(int)):.3f}")
        print(f"  Correlation (Proximity, P&L): {prox_df['Proximity'].corr(prox_df['PnL']):.3f}")
    else:
        print("No proximity data available")

## 8. Proximity Threshold Testing

In [ ]:
# Test different minimum proximity thresholds
proximity_results = []

for min_prox in [0, 30, 50, 70]:
    test_params = VCP_PARAMS['current'].copy()
    test_params['min_proximity_score'] = min_prox
    
    print(f"Testing min_proximity_score = {min_prox}...")
    
    result = backtester.run_backtest(
        symbols=TEST_SYMBOLS,
        strategy_name=f"VCP_Prox{min_prox}",
        params=test_params,
        risk_params=RISK_PARAMS['current'],
        exit_method='fixed_target',
        start_date='2023-01-01'
    )
    
    proximity_results.append({
        'Min Proximity': min_prox,
        'Trades': result.total_trades,
        'Win Rate %': result.win_rate,
        'Avg Win %': result.avg_win_pct,
        'Avg Loss %': result.avg_loss_pct,
        'Profit Factor': result.profit_factor,
        'Avg R:R': result.avg_rr_realized,
    })

prox_results_df = pd.DataFrame(proximity_results)
print("\n" + "=" * 80)
print("PROXIMITY THRESHOLD COMPARISON")
print("=" * 80)
display(prox_results_df)

In [ ]:
# Visualize proximity threshold impact
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

x = prox_results_df['Min Proximity']

axes[0].bar(x.astype(str), prox_results_df['Trades'], color='steelblue', edgecolor='black')
axes[0].set_xlabel('Min Proximity Threshold')
axes[0].set_ylabel('Number of Trades')
axes[0].set_title('Trade Count by Proximity Filter')

axes[1].bar(x.astype(str), prox_results_df['Win Rate %'], color='green', edgecolor='black')
axes[1].set_xlabel('Min Proximity Threshold')
axes[1].set_ylabel('Win Rate %')
axes[1].set_title('Win Rate by Proximity Filter')

axes[2].bar(x.astype(str), prox_results_df['Profit Factor'], color='purple', edgecolor='black')
axes[2].set_xlabel('Min Proximity Threshold')
axes[2].set_ylabel('Profit Factor')
axes[2].set_title('Profit Factor by Proximity Filter')

plt.tight_layout()
plt.show()

---
# PHASE 3: RS Rating Relaxation
---

## 9. RS Threshold Sensitivity

In [ ]:
# Test different RS thresholds
rs_results = []

for rs_min in [70, 75, 80, 85, 90, 95]:
    test_params = VCP_PARAMS['current'].copy()
    test_params['rs_rating_min'] = rs_min
    
    print(f"Testing rs_rating_min = {rs_min}...")
    
    result = backtester.run_backtest(
        symbols=TEST_SYMBOLS,
        strategy_name=f"VCP_RS{rs_min}",
        params=test_params,
        risk_params=RISK_PARAMS['current'],
        exit_method='fixed_target',
        start_date='2023-01-01'
    )
    
    rs_results.append({
        'RS Min': rs_min,
        'Trades': result.total_trades,
        'Win Rate %': result.win_rate,
        'Profit Factor': result.profit_factor,
        'Avg R:R': result.avg_rr_realized,
        'Total Return %': result.total_return_pct,
    })

rs_results_df = pd.DataFrame(rs_results)
print("\n" + "=" * 80)
print("RS THRESHOLD COMPARISON")
print("=" * 80)
display(rs_results_df)

In [ ]:
# RS visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

x = rs_results_df['RS Min']

axes[0, 0].plot(x, rs_results_df['Trades'], marker='o', linewidth=2)
axes[0, 0].set_xlabel('RS Min Threshold')
axes[0, 0].set_ylabel('Number of Trades')
axes[0, 0].set_title('Trade Count vs RS Threshold')
axes[0, 0].grid(True)

axes[0, 1].plot(x, rs_results_df['Win Rate %'], marker='o', linewidth=2, color='green')
axes[0, 1].set_xlabel('RS Min Threshold')
axes[0, 1].set_ylabel('Win Rate %')
axes[0, 1].set_title('Win Rate vs RS Threshold')
axes[0, 1].grid(True)

axes[1, 0].plot(x, rs_results_df['Profit Factor'], marker='o', linewidth=2, color='purple')
axes[1, 0].set_xlabel('RS Min Threshold')
axes[1, 0].set_ylabel('Profit Factor')
axes[1, 0].set_title('Profit Factor vs RS Threshold')
axes[1, 0].axhline(y=1.5, color='red', linestyle='--', label='Target: 1.5')
axes[1, 0].legend()
axes[1, 0].grid(True)

axes[1, 1].plot(x, rs_results_df['Total Return %'], marker='o', linewidth=2, color='orange')
axes[1, 1].set_xlabel('RS Min Threshold')
axes[1, 1].set_ylabel('Total Return %')
axes[1, 1].set_title('Total Return vs RS Threshold')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

---
# PHASE 4: Stop/Target Optimization (R:R ≥ 3.0)
---

## 10. Fixed Target Combinations

In [ ]:
# Test stop/target combinations with R:R >= 3.0
rr_results = []

combinations = [
    (6, 18), (6, 21), (6, 24),  # 6% stop
    (7, 21), (7, 24), (7, 28),  # 7% stop
    (8, 24), (8, 28), (8, 32),  # 8% stop
]

for stop, target in combinations:
    rr = target / stop
    risk_params = {
        'default_stop_loss_pct': stop,
        'default_target_pct': target,
    }
    
    print(f"Testing Stop={stop}%, Target={target}% (R:R={rr:.2f})...")
    
    result = backtester.run_backtest(
        symbols=TEST_SYMBOLS,
        strategy_name=f"VCP_S{stop}_T{target}",
        params=VCP_PARAMS['current'],
        risk_params=risk_params,
        exit_method='fixed_target',
        start_date='2023-01-01'
    )
    
    rr_results.append({
        'Stop %': stop,
        'Target %': target,
        'Theoretical R:R': rr,
        'Trades': result.total_trades,
        'Win Rate %': result.win_rate,
        'Realized R:R': result.avg_rr_realized,
        'Profit Factor': result.profit_factor,
        'Total Return %': result.total_return_pct,
    })

rr_results_df = pd.DataFrame(rr_results)
print("\n" + "=" * 80)
print("STOP/TARGET COMBINATIONS (R:R ≥ 3.0)")
print("=" * 80)
display(rr_results_df.sort_values('Profit Factor', ascending=False))

In [ ]:
# Heatmap of profit factor by stop/target
pivot_pf = rr_results_df.pivot(index='Stop %', columns='Target %', values='Profit Factor')
pivot_wr = rr_results_df.pivot(index='Stop %', columns='Target %', values='Win Rate %')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(pivot_pf, annot=True, fmt='.2f', cmap='RdYlGn', ax=axes[0])
axes[0].set_title('Profit Factor by Stop/Target')

sns.heatmap(pivot_wr, annot=True, fmt='.1f', cmap='RdYlGn', ax=axes[1])
axes[1].set_title('Win Rate % by Stop/Target')

plt.tight_layout()
plt.show()

---
# PHASE 5: Fixed Target vs Trailing Stop
---

## 11. Trailing Stop Configurations

In [ ]:
# Test trailing stop configurations
trailing_results = []

configs = [
    (7, 8, 5),   # Standard
    (7, 10, 5),  # Later activation
    (7, 8, 6),   # Wider trail
    (8, 8, 5),   # 8% initial
    (8, 10, 6),  # Conservative
    (7, 12, 7),  # Let winners run
]

for stop, activation, trail in configs:
    risk_params = {
        'default_stop_loss_pct': stop,
        'trailing_stop_activation_pct': activation,
        'trailing_stop_distance_pct': trail,
    }
    
    print(f"Testing Trailing: Stop={stop}%, Activate=+{activation}%, Trail={trail}%...")
    
    result = backtester.run_backtest(
        symbols=TEST_SYMBOLS,
        strategy_name=f"VCP_Trail_S{stop}_A{activation}_T{trail}",
        params=VCP_PARAMS['current'],
        risk_params=risk_params,
        exit_method='trailing_stop',
        start_date='2023-01-01'
    )
    
    trailing_results.append({
        'Initial Stop %': stop,
        'Activation %': activation,
        'Trail %': trail,
        'Trades': result.total_trades,
        'Win Rate %': result.win_rate,
        'Avg Win %': result.avg_win_pct,
        'Avg Loss %': result.avg_loss_pct,
        'Realized R:R': result.avg_rr_realized,
        'Profit Factor': result.profit_factor,
        'Avg Days': result.avg_days_held,
    })

trailing_df = pd.DataFrame(trailing_results)
print("\n" + "=" * 80)
print("TRAILING STOP CONFIGURATIONS")
print("=" * 80)
display(trailing_df.sort_values('Profit Factor', ascending=False))

In [ ]:
# Compare best fixed target vs best trailing stop
best_fixed = rr_results_df.loc[rr_results_df['Profit Factor'].idxmax()]
best_trailing = trailing_df.loc[trailing_df['Profit Factor'].idxmax()]

comparison = pd.DataFrame([
    {
        'Method': 'Best Fixed Target',
        'Config': f"Stop {best_fixed['Stop %']}% / Target {best_fixed['Target %']}%",
        'Win Rate %': best_fixed['Win Rate %'],
        'Realized R:R': best_fixed['Realized R:R'],
        'Profit Factor': best_fixed['Profit Factor'],
    },
    {
        'Method': 'Best Trailing Stop',
        'Config': f"Stop {best_trailing['Initial Stop %']}% / Act +{best_trailing['Activation %']}% / Trail {best_trailing['Trail %']}%",
        'Win Rate %': best_trailing['Win Rate %'],
        'Realized R:R': best_trailing['Realized R:R'],
        'Profit Factor': best_trailing['Profit Factor'],
    }
])

print("=" * 80)
print("FIXED TARGET vs TRAILING STOP - BEST CONFIGURATIONS")
print("=" * 80)
display(comparison)

---
# PHASE 6: Summary & Recommendations
---

## 12. Final Summary

In [ ]:
# Generate final summary
print("=" * 80)
print("BACKTEST SUMMARY & RECOMMENDATIONS")
print("=" * 80)

print("\n1. BASELINE PERFORMANCE:")
print(f"   VCP: {vcp_baseline.total_trades} trades, {vcp_baseline.win_rate:.1f}% WR, {vcp_baseline.profit_factor:.2f} PF")
print(f"   Pivot: {pivot_baseline.total_trades} trades, {pivot_baseline.win_rate:.1f}% WR, {pivot_baseline.profit_factor:.2f} PF")

print("\n2. PROXIMITY EFFECT:")
if len(prox_results_df) > 0:
    print(f"   No filter: {prox_results_df.iloc[0]['Trades']} trades, {prox_results_df.iloc[0]['Win Rate %']:.1f}% WR")
    if len(prox_results_df) > 3:
        print(f"   Prox ≥ 70: {prox_results_df.iloc[3]['Trades']} trades, {prox_results_df.iloc[3]['Win Rate %']:.1f}% WR")

print("\n3. BEST CONFIGURATIONS:")
print(f"   Fixed Target: Stop {best_fixed['Stop %']}%, Target {best_fixed['Target %']}% -> PF {best_fixed['Profit Factor']:.2f}")
print(f"   Trailing Stop: Stop {best_trailing['Initial Stop %']}%, Activate +{best_trailing['Activation %']}% -> PF {best_trailing['Profit Factor']:.2f}")

print("\n4. RS RELAXATION:")
if len(rs_results_df) > 0:
    rs90 = rs_results_df[rs_results_df['RS Min'] == 90].iloc[0] if 90 in rs_results_df['RS Min'].values else None
    rs85 = rs_results_df[rs_results_df['RS Min'] == 85].iloc[0] if 85 in rs_results_df['RS Min'].values else None
    if rs90 is not None and rs85 is not None:
        trade_change = ((rs85['Trades'] - rs90['Trades']) / rs90['Trades'] * 100) if rs90['Trades'] > 0 else 0
        wr_change = rs85['Win Rate %'] - rs90['Win Rate %']
        print(f"   RS 90 → 85: {trade_change:+.0f}% trades, {wr_change:+.1f}% win rate")

In [ ]:
# Save all results to CSV for further analysis
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if len(rr_results_df) > 0:
    rr_results_df.to_csv(f'results_fixed_target_{timestamp}.csv', index=False)
if len(trailing_df) > 0:
    trailing_df.to_csv(f'results_trailing_stop_{timestamp}.csv', index=False)
if len(rs_results_df) > 0:
    rs_results_df.to_csv(f'results_rs_sensitivity_{timestamp}.csv', index=False)
if len(prox_results_df) > 0:
    prox_results_df.to_csv(f'results_proximity_{timestamp}.csv', index=False)

print(f"\nResults saved with timestamp: {timestamp}")
print("\nBacktest complete!")